### Librerias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [2]:
# Configurar la visualización de gráficos inline
%matplotlib inline

## csv 1


### importacion de CSV

### Cantidad de variables por tipo de datos

### Conversion de tipos de datos de las columnas

### Codificacion

### Escalamiento

### Proporcion entre variables categoricas

### Estadisticas generales

### Correlacion

### Guardado de CSV

## csv 2

### Importacion de CSV

### Cantidad de variables por tipo de datos

### Proporcion entre variables categoricas

### Codificacion

### Escalamiento

### Estadisticas basicas

### Correlacion

### Guardado de CSV